### **Load Environment variables from .env file**

In [8]:
from langchain.llms import AzureOpenAI
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown
import json

In [27]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

OPENAI_DAVINCI_DEPLOYMENT_NAME = os.getenv("OPENAI_DAVINCI_DEPLOYMENT_NAME")
OPENAI_DAVINCI_MODEL_NAME = os.getenv("OPENAI_DAVINCI_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

### **Model initialization**

In [32]:
def init_llm(model=OPENAI_DAVINCI_MODEL_NAME,
             deployment_name=OPENAI_DAVINCI_DEPLOYMENT_NAME, 
             temperature=0,
             max_tokens=4000,
             stop="<|im_end|>", 
             ):
    
    llm = AzureOpenAI(deployment_name=deployment_name,  
                  model=model,
                  temperature=temperature,) 
    return llm


### **Prompt engineering techniques**
Those techniques are not specific to summarization, but can be used for any task.

1. Use delimeters to clearly separate the exact text the model should summarize.

    Delimiters could be any kind of punctuation, that separates specific pieces of text. 
    Tripple quotes, Triple backtics, Triple dashes, Angle brackets, XML tags, etc.

2. Ask model for structured output, e.g json, html, etc. If you ask for a json define the structure of the json, e.g. what fields should be in the json.

3. Check assumptions required to do the task.

4. Few-shot prompting. Provide examples of completing tasks and then ask model to perform the task.



In [33]:
# Using delimiters to clearly separate input text.

text = f"""
**Customer**: Hello, my name is John, and I'm a customer of Imaginal Bank.
**Clerk**: Hello, John! My name is Sara, and I'm a customer service representative at Imaginal Bank. How can I assist you today?
**Customer**: Hi, Sara. I'm interested in your bank's investment programs. 
              Can you tell me more about them, especially in terms of risk management?

**Clerk**: Absolutely, John. We have a few key programs I can highlight.

First, there's our 'Balanced Growth Fund'. It's a diversified mutual fund that invests in a mix of equities and bonds to provide both growth and income, reducing risk through diversification. 

We also have the 'Index Tracker ETF', which is designed to replicate the performance of a specific market index. By spreading investments across the entire index, it inherently reduces the risk associated with individual stocks.

Additionally, for those with a lower risk tolerance, we have the 'Secure Income Bond Fund', which focuses on government and high-quality corporate bonds. 

Our financial advisors are always available to guide you in choosing the right program based on your financial goals and risk tolerance.

**Customer**: I see. Could you elaborate on how the Balanced Growth Fund manages risk?

**Clerk**: Sure. The Balanced Growth Fund mitigates risk by diversifying investments across a wide range of assets. If one investment performs poorly, it's likely to be offset by other investments that are performing well. Furthermore, our portfolio managers actively manage the fund, adjusting holdings based on changing market conditions to manage risk and enhance returns.

**Customer**: Does the bank provide any tools to monitor my investments?

**Clerk**: Yes, John. We offer an online platform called 'Imaginal Investor Dashboard'. It provides real-time tracking of your investments, balance updates, and market trends. You can also set up alerts to be notified about significant changes in your portfolio.

**Customer**: That sounds quite comprehensive. How can I get started?

**Clerk**: You can schedule an appointment with one of our financial advisors. They'll walk you through your options, help you understand your risk tolerance, and guide you in choosing the right investment program. Would you like me to arrange that for you?

**Customer**: Yes, please. That would be helpful.

**Clerk**: Fantastic, John! Let's get that set up for you...

"""

prompt = f""" Summarize the text delimited by triple backticks into single sentence. 
```{text}```"""

llm=init_llm()
sum = llm(prompt)
display(Markdown(sum))



Imaginal Bank offers a variety of investment programs with risk management strategies, as well as an online platform to monitor investments, and financial advisors to help customers choose the right program.

In [34]:
# Check the specific conditions/assumptions in the text

prompt = f""" You will be provided with a text delimited by triple quotes. 
If it's contains utterances of a Customer and a Clerk, summarize the text into single sentence.
If the text does not contain utterances of a Customer and a Clerk, 
then just write 'No customer and clerk utterances found'.
\"\"\" {text}  \"\"\"
"""
llm=init_llm()
sum = llm(prompt)
display(Markdown(sum))
 

John, a customer of Imaginal Bank, asked Sara, a customer service representative, about the bank's investment programs and was provided with information about the Balanced Growth Fund, Index Tracker ETF, and Secure Income Bond Fund, as well as the Imaginal Investor Dashboard, and was offered an appointment with a financial advisor.

In [35]:
# Check the specific conditions/assumptions in the text

text = f"""just a text"""

prompt = f""" You will be provided with a text delimited by triple quotes. If it's contains utterances of a Customer and a Clerk, summarize the text into single sentence. If the text does not contain utterances of a Customer and a Clerk, 
then just write 'No customer and clerk utterances found'.
\"\"\" {text}  \"\"\"
"""
llm=init_llm()
sum = llm(prompt)
display(Markdown(sum))

No customer and clerk utterances found.

#### Give the model time to think

Very useful technique is to give the model time to think by specifying the strict steps (sub-tasks) to complete the task and asking for output in a specific format. This is a very powerful technique.


In [15]:
text = f"""
**Customer**: Hello, my name is John, and I'm a customer of Imaginal Bank.
**Clerk**: Hello, John! My name is Sara, and I'm a customer service representative at Imaginal Bank. How can I assist you today?
**Customer**: Hi, Sara. I'm interested in your bank's investment programs. 
              Can you tell me more about them, especially in terms of risk management?

**Clerk**: Absolutely, John. We have a few key programs I can highlight.

First, there's our 'Balanced Growth Fund'. It's a diversified mutual fund that invests in a mix of equities and bonds to provide both growth and income, reducing risk through diversification. 

We also have the 'Index Tracker ETF', which is designed to replicate the performance of a specific market index. By spreading investments across the entire index, it inherently reduces the risk associated with individual stocks.

Additionally, for those with a lower risk tolerance, we have the 'Secure Income Bond Fund', which focuses on government and high-quality corporate bonds. 

Our financial advisors are always available to guide you in choosing the right program based on your financial goals and risk tolerance.

**Customer**: I see. Could you elaborate on how the Balanced Growth Fund manages risk?

**Clerk**: Sure. The Balanced Growth Fund mitigates risk by diversifying investments across a wide range of assets. If one investment performs poorly, it's likely to be offset by other investments that are performing well. Furthermore, our portfolio managers actively manage the fund, adjusting holdings based on changing market conditions to manage risk and enhance returns.

**Customer**: Does the bank provide any tools to monitor my investments?

**Clerk**: Yes, John. We offer an online platform called 'Imaginal Investor Dashboard'. It provides real-time tracking of your investments, balance updates, and market trends. You can also set up alerts to be notified about significant changes in your portfolio.

**Customer**: That sounds quite comprehensive. How can I get started?

**Clerk**: You can schedule an appointment with one of our financial advisors. They'll walk you through your options, help you understand your risk tolerance, and guide you in choosing the right investment program. Would you like me to arrange that for you?

**Customer**: Yes, please. That would be helpful.

**Clerk**: Fantastic, John! Let's get that set up for you...

"""

prompt = f""" Your task is to perform the following actions:
1 - Summarize the text delimited by triple backticks into single sentence.
2 - Translate the summary into Portuguese.
3 - List the Customer questions.

Use the following output format:
Summary: <summary>
Translated summary: <portuguese summary>
Customer questions: <enumerated customer questions>

Text: ```{text}```
"""

llm=init_llm()
sum = llm(prompt)
display(Markdown(sum))



Summary: Imaginal Bank offers a variety of investment programs with different levels of risk management, as well as an online platform to monitor investments, and financial advisors to help customers choose the right program.
Translated summary: A Imaginal Bank oferece uma variedade de programas de investimento com diferentes níveis de gerenciamento de risco, bem como uma plataforma online para monitorar investimentos e consultores financeiros para ajudar os clientes a escolher o programa certo.
Customer questions: 
1. Can you tell me more about Imaginal Bank's investment programs, especially in terms of risk management?
2. Could you elaborate on how the Balanced Growth Fund manages risk?
3. Does the bank provide any tools to monitor my investments?
4. How can I get started?

##### Let's do some math

In [16]:
prompt = f""" Determine id the sudent's solution is correct or not.
Question: When I was 2 years old my sister was twice my age. I'm now 40 years old how old is my sister now? 

Student's answer: The sister is now 80 years old.
"""

llm=init_llm()
sum = llm(prompt)
display(HTML(sum))

##### Ask model to do its own solution and then to compare both solutions and coclude which is correct.
 Define the task as a list of sub-tasks and ask the model to perform them in a specific order (splitting the task into sub-tasks technique). 
 Then ask the model to compare its own solution with the solution provided by the model and conclude which is correct.
 Ask model to share it reasoning for the conclusion.

In [43]:
Question = f"""When I was 2 years old my sister was twice of my age . I'm now 40 years old, how old is my sister now?"""
Student_Solution = f""" The sister is now 80 years old."""

prompt = f""" Determine if the Student's Solution for the Question is correct or not.
To solve the problem do the following:
1 - First, work out your OWN solution to the problem. Evaluate your final result to make sure it is correct and adheres to the question's conditions. 
2 - Second, compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.


Use the following output format:
Actual solution steps: <your own solution steps>
Student's solution: <student's solution>
Student's solution is correct: <true/false>


Student's Solution: ```{Student_Solution}```
Question: ```{Question}```

"""

llm=init_llm()
sum = llm(prompt)
display(Markdown(sum))


Actual solution steps: 
1. When the student was 2 years old, their sister was twice their age, so their sister was 4 years old. 
2. Since the student is now 40 years old, their sister has aged 40 years since then. 
3. Therefore, their sister is now 40 + 4 = 44 years old. 

Student's solution: The sister is now 80 years old.
Student's solution is correct: False

#### Model Hallucinations and how to avoid them

Hallucination is when the model generates text that is not supported by the input.
To reduce hallucinations, use the techniques listed above, and strictly instruct the model to find the relevant information in the input text. 
If the information is not in the input, instruct the model to generate a specific output, e.g. "I don't know the answer to this question". 

#### Iterative Prompt Development
Prompt engineering is an iterative process. You almost never get the prompt right the first time.
You start with the idea, then you implement the prompt, get the experimental resuslts, do the Error Analysis and iterate again.

**Try -> Analyze -> Clarify Instructions -> Try again**

In more advanced phases refine prompts with a batch of examples.

### Basic Summarization 

In [45]:
#helper function
#Note that this functions calculates the number of tokens in the prompt
def summarize_text(llm, prompt_prefix, text_file):
    
    with open(text_file, 'r') as file:
        # read the entire file into a string
        data = file.read()

    # concatenate the prompt with the data
    prompt = prompt_prefix + data
    #check the number of tokens in the prompt
    num_tokens = llm.get_num_tokens(prompt)
    print (f"Number of tokens in final prompt is: {num_tokens}")
    return llm(prompt)

In [47]:
llm=init_llm()
prompt_prefix = "Summarize the text below for a call center supervisor:\n"

sum = summarize_text(llm, prompt_prefix, "./data/bank-call-center-transcript.txt")
display(Markdown(sum))

Number of tokens in final prompt is: 554



John, a customer of Imaginal Bank, is interested in the bank's investment programs and inquires about risk management. The customer service representative explains the three main programs offered by the bank and how they manage risk. The representative also mentions the Imaginal Investor Dashboard, an online platform that provides real-time tracking of investments, balance updates, and market trends. Finally, the representative offers to arrange an appointment with a financial advisor to help John choose the right investment program.

#### More advanced prompts

In [51]:
llm=init_llm()
prompt_prefix = """ Prepare a summary for the call center supervisor based on the points mentioned below. 
Please evaluate whether the clerk successfully accomplished the following tasks:

Greeting the customer politely and professionally.
Accurately understanding the customer's inquiry.
Providing clear and detailed information in response.
Asking questions as needed for clarification.
Discussing both benefits and risks with the customer.
Explaining the tools and resources available to the customer.
Inviting the customer to take further action.
Offering assistance for the next steps.
Ending the conversation on a positive note.


"""

sum = summarize_text(llm, prompt_prefix, "./data/bank-call-center-transcript.txt")
display(Markdown(sum))

Number of tokens in final prompt is: 663



The customer service representative, Sara, successfully accomplished all the tasks. She greeted the customer politely and professionally, accurately understood the customer's inquiry, provided clear and detailed information in response, asked questions as needed for clarification, discussed both benefits and risks with the customer, explained the tools and resources available to the customer, invited the customer to take further action, offered assistance for the next steps, and ended the conversation on a positive note.

##### Output in tabular format

In [52]:
llm=init_llm()
prompt_prefix = """Summarize the following information for the attention of the 
call center supervisor.
Present the output in the form of an HTML table, with each item on a separate row.

Assign a color code to each item based on the clerk's performance - 
items that were successfully addressed should be marked in green, 
whereas items that were not met should be highlighted in red.

Here are the evaluation criterias to consider:

Did the clerk greet the customer in a polite and professional manner?
Did the clerk begin the conversation on a negative note?
Did the clerk understand the customer's inquiry?
Did the clerk provide clear and detailed information?
Did the clerk ask questions to clarify the situation?


"""

res = summarize_text(llm, prompt_prefix, "./data/bank-call-center-transcript.txt")
display(HTML(res.strip()))

Number of tokens in final prompt is: 695


##### Output as JSON

In [53]:
llm=init_llm()
prompt_prefix = """"Prepare a summary of the following points for the call center supervisor. 
The output should be presented in JSON format, adhering to the following schema:

{
    "Greet the Customer Politely and Professionally": "Yes/No",
    "Understand the Customer's Inquiry": "Yes/No",
    "Provide Clear and Detailed Information": "Yes/No",
    "Ask Clarifying Questions": "Yes/No",
    "Discuss the Benefits and Risks": "Yes/No",
    "Explain Available Tools and Resources": "Yes/No",
    "Invite Further Action": "Yes/No",
    "Offer to Assist with Next Steps": "Yes/No",
    "End on a Positive Note": "Yes/No"
}

Please evaluate the clerk's performance based on the following points:

Did the clerk greet the customer in a polite and professional manner?
Did the clerk comprehend the customer's inquiry accurately?
Did the clerk provide comprehensive and clear information?
Did the clerk ask relevant questions to clarify the customer's situation?
Did the clerk explain the benefits and potential risks to the customer?
Did the clerk detail the tools and resources available to the customer?
Did the clerk encourage the customer to take further action?
Did the clerk offer assistance with proceeding to the next steps?
Did the clerk end the interaction on a positive and upbeat note?"
"""

res = summarize_text(llm, prompt_prefix, "./data/bank-call-center-transcript.txt")
display(JSON(json.loads(res)))


Number of tokens in final prompt is: 847


<IPython.core.display.JSON object>

### Summarize large documents
To summarize larger documents, we can split the document into smaller chunks and summarize each chunk separately. We can then combine the summaries of each chunk to get the final summary.
LangChain has a built-in chain for doing that. 



In [54]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [55]:
from langchain.document_loaders import PyPDFLoader
llm=init_llm()

large_pdf_path ="./data/Large_language_model.pdf"
loader = PyPDFLoader(large_pdf_path)

#output type is List[Document]
pages = loader.load_and_split()

#count tokens in the document
total_tokens = 0

for page in pages:
    total_tokens += llm.get_num_tokens(page.page_content)
    
# The document has more 14000 tokens. This is too many for the LLM to process in one go.
# This is whay we split the document into smaller chunks.
print(f"Total tokens in the document: {total_tokens}")


Total tokens in the document: 14386


In [57]:
#create summarization chain
summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce',verbose=False )

##### There are following chain_types: stuff, map_reduce, refine, map-rerank
See here for the details: https://docs.langchain.com/docs/components/chains/index_related_chains

In [58]:
#Many calls in loop causing RateLimitError: Too Many Requests
#sum = summary_chain.run(pages)
sum = summary_chain.run(pages[0:5])

display(Markdown(sum))

 Large language models (LLMs) are neural networks with billions of parameters, trained on large datasets of unlabeled text. They are used for a wide range of tasks, such as sentiment analysis, named-entity recognition, and part-of-speech tagging. LLMs are pre-trained on large datasets, and their performance is related to the size of the model, the size of the training dataset, the cost of training, and the performance after training. Instruction tuning and reinforcement learning from human feedback (RLHF) are used to fine-tune and train the LLM. Evaluation of LLMs is done by measuring their perplexity on a test set of unseen data.